# Exercise 1.1


Installing libraries: 

In [27]:
%pip install opencv-python numpy sys

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sys (from versions: none)
ERROR: No matching distribution found for sys

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import libraries :

In [28]:
import numpy as np
import sys
import cv2

In [29]:
# The higher the value, the bigger a moving object should be to be detected
minimun_area_contour = 4000

In [30]:
# Open the video file named "Traffic_Laramie_1.mp4":
video_capture = cv2.VideoCapture("Traffic_Laramie_1.mp4")

# Ensure the video file is accessible:
assert video_capture.isOpened(), "Video file could not be opened"

# Set up a background remover using a Gaussian mixture model:
background_remover = cv2.createBackgroundSubtractorKNN(detectShadows=False, history=10000, dist2Threshold=900)

# Create a window to display the live video:
cv2.namedWindow("Live Feed")

# Process each frame of the video:
while True:
    # Capture the next frame:
    ret, frame = video_capture.read()

    # Exit the loop if there are no more frames:
    if not ret:
        break

    # Smooth out the frame to reduce noise:
    smoothed_frame = cv2.GaussianBlur(frame, (5, 5), 0)

    # Separate the foreground from the background:
    foreground_mask = background_remover.apply(smoothed_frame)

    # Convert the foreground mask to black and white, minimizing shadows:
    _, foreground_mask = cv2.threshold(foreground_mask, 100, 255, cv2.THRESH_BINARY)

    # Enhance the mask's clarity using image processing techniques:
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find the outlines of detected objects in the mask:
    object_contours, _ = cv2.findContours(foreground_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out contours that are likely too small to be objects:
    object_contours = [contour for contour in object_contours if cv2.contourArea(contour) > minimun_area_contour]

    # Focus on objects in the lower half of the frame:
    object_contours = [contour for contour in object_contours if cv2.boundingRect(contour)[1] > frame.shape[0] / 2]

    # Simplify the contours without holes:
    object_contours = [cv2.convexHull(contour) for contour in object_contours]

    # Draw contours for visual inspection:
    cv2.drawContours(frame, object_contours, -1, (0, 255, 0), 2)

    # Prepare containers to store bounding boxes around detected objects:
    bounding_boxes = []

    # Determine the rectangular coordinates of the contours:
    for contour in object_contours:
        (x, y, width, height) = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x + width, y + height])

    # Draw visible rectangles around the detected objects:
    for [x, y, x2, y2] in bounding_boxes:
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 1)

    # Show the processed frame in the "Live Feed" window:
    cv2.imshow("Live Feed", frame)

    # Allow the user to exit the loop by pressing "q":
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyWindow("Live Feed")
        cv2.waitKey(1)
        break
